# Семинар 10. Доверительные интервалы и тестирование параметрических гипотез

In [ ]:
import scipy.stats as stats # импортируем библиотеку для статистических функций
import pandas as pd # импортируем библиотеку pandas для обработки табличных данных
import numpy as np # импортируем библиотеку numpy для реализации математических операций

Задача этого семинара -- научиться по выборкам делать выводы о параметрах генеральной совокупности. 


Проведем эксперимент: возьмем выборку объемом в 10, 20 и 30 чисел из нормальной генеральной совокупности с матожиданием, равным нулю, и стандартным отклонением, равным 1 (стандартный номальный закон $Z_i \sim N(0;1) $). 
Если бы мы не знали, что это стандартный нормальный закон, то  для того, чтобы оценить, чему равно среднее для генеральной совокупности, нам надо было бы посчитать выборочное среднее по каждой из трех выборок. 

In [ ]:
for i in range(3): # реализуем три итерации цикла
      print(np.random.normal(size=(10*(i+1))).mean()) # на каждой берем выборку и считаем среднее

0.32396171081950154
-0.2907445715619434
0.3904789953587259


Все три раза они получились разными (не равными друг другу), и к тому же не равными настоящему среднему из генеральной совокупности (нулю).

Выборочное среднее, которое мы три раза искали выше, называется "точечной оценкой" генерального среднего значения, и мы убедились в том, что такая оценка близка к оцениваемому параметру (обычно чем больше объем выборки, тем ближе), но никогда не равна ему. Это связано с тем, что выборочное среднее -- это случайная величина, которая сама имеет нормальное распределение, а для непрерывной случайной величины вероятность того, что она примет какое-то конкретное значение, равна нулю.

Чтобы обойти эту проблему с точечными оценками, можно использовать "интервальные" оценки -- сообщать некоторый диапазон значений, который будет содержать оцениваемый параметр с нужной заранее известной вероятностью. 

Из курса статистики известно, что доверительный интервал, который с доверительной вероятностью $\gamma$ накроет генеральное среднее, ищется по формуле $M \pm \varepsilon$ (в наших обозначениях $M = \bar{X}$, то есть это выборочное среднее), а для нахождения  $\varepsilon$ надо использовать одну из двух формул:

1. Если нам известно стандартное отклонение генеральной совокупности $\sigma$, то используем формулу $\varepsilon = z_{\alpha/2}\cdot \frac{\sigma}{\sqrt n}$, где $z_{\alpha/2}$ отрезает от нормального распределения правый хвост объема $\alpha/2$.

2. Если же у нас нет генерального стандартного отклонения, то вместо $\sigma$ надо использовать $s$  -- выборочное стандартное оклонение, а вместо $z_{\alpha/2}$ используется $t_{\alpha/2}$ -- это число отрезает от распределения Стьюдента правый хвост объема $\alpha/2$, распределение используется с $n-1$ степенью свободы. 

Уровень доверия $\gamma$ берется близким к 1 (0.9, 0.95, 0.99), а $\alpha = 1 - \gamma$



Теперь посмотрим, как найти доверительный интервал с помощью встроенных методов питона.

Предположим, мы хотим найти 95% доверительный интервал для средней оценки за контрольную работу по 9 студентам, если нам заранее известно, что стандартное отклонение  оценки за эту контрольную работу по всем студентам равно $1.5$.

In [ ]:
data1 = np.array([6, 7, 8, 4, 3, 6, 7, 8, 5])
sigma = 1.5
stats.norm.interval(0.95, loc=data1.mean(), scale=sigma/np.sqrt(len(data1)))
# первый параметр это доверительная вероятность, второй - середина доверительного интервала, 
# третий - соответствующий множитель из формулы

(5.020018007729973, 6.979981992270027)

Вывод: с доверительной вероятностью 95% интервал (5.02, 6.98) накрывает среднюю оценку за контрольную по всем студентам.

Теперь сделаем все то же самое для второго типа доверительных интервалов, то есть в ситуации, когда у нас нет генерального стандартного отклонения (а скорее всего так и будет - довольно редко бывает так, что генеральное стандартное отклонение есть, а генерального среднего нет).

Пусть у нас есть оценки за экзамен по 10 студентам, и мы снова хотим найти 95% доверительный интервал для средней оценки по всем студентам (генеральное стандартное отклонение мы не знаем)

In [ ]:
data2 = np.array([6, 8, 5, 4, 7, 5, 8, 7, 6, 9])
sigma = data2.std(ddof=1) # т.к это не генеральное стандартное отклонение, то делим не на n, а на n-1
stats.t.interval(0.95, df=len(data2) - 1, loc=data2.mean(), scale=sigma/np.sqrt(len(data2)))
# добавился новый параметр - это число степеней свободы, то есть n-1

(5.368921418629505, 7.631078581370495)

Вывод: с доверительной вероятностью 95% интервал (5.37, 7.63) накрывает среднюю оценку за экзамен по всем студентам.

Пример того, как ищутся критические точки $z_{\alpha/2}$ и $t_{\alpha/2}$ для 95% доверительных интервалов приведен ниже 

In [ ]:
gamma = 0.95
alpha = 1 - gamma

In [ ]:
stats.norm.ppf(1 - alpha/2) # ppf  - функция процентных точек, находит квантиль указанного в скобках уровня

1.959963984540054

In [ ]:
stats.t.ppf(1 - alpha/2, df=1000) # число степеней свободы должно равняться n-1

1.9623390808264074

# Параметрические критерии для тестирования гипотез
Выше мы вспомнили, как искать интервальные оценки для среднего в генеральной совокупности, теперь мы подойдем к задаче оценки параметров генеральной совокупности с новой стороны. Речь все так же будет идти о генеральном среднем.

Предположим, у нас есть все та же контрольная, о которой мы говорили в первом примере, и до контрольной у нас есть четыре предположения о том, насколько сложной будет работа: кто-то считает, что средний балл будет равен 4.2, кто-то считает что 4.9, еще есть вариант 6.2 и вариант 7.4. 

Мы строили 95% доверительный интервал, который оказался равен (5.02; 6.98) -- мы на 95% уверены, что этот интервал содержит среднюю оценку по всем студентам, это означает, что предположение о том, что средняя оценка равна 6.2 не противоречит нашим реальным данным, а оставшиеся три, которые не попадают в интервал, следует поставить под сомнение.

Примерно так и будет проходить проверка гипотез: у нас есть "нулевая" или "основная" гипотеза - предположение о среднем значении интересующего нас признака. По  имеющейся случайной выборке мы считаем число, которое называется "статистикой". Если предположение нулевой гипотезы верное, то статистика имеет известное распределение, и можно понять, насколько полученное число соответствует этому распределению. Например, для стандартного нормального закона значение -0.2 не вызывает никаких вопросов, а 4.5 вызывает -- если мы увидим такое число, то возможны два варианта: либо произошло очень маловероятное событие, либо мы неправильно предположили, что работаем со стандартным нормальным законом.

Для того, чтобы отделить критические значения статистики от естественных, задается малая вероятность $\alpha$ (уровень значимости). Это вероятность ошибки первого рода, то есть того, что мы посчитаем нулевую гипотезу неверной, хотя на самом деле она будет верной.

# Z-тест
У нас есть нулевая гипотеза

$H_0: \mu = \mu_0$

(гипотеза о том, чему равно матожидание признака из нормальной генеральной совокупности)

У нас есть альтернативная гипотеза (двусторонняя или односторонняя), и если основная гипотеза будет отвергнута, то мы примем альтернативную

$H_1: \mu \ne \mu_0$  

 либо   $H_1: \mu > \mu_0$ 
 
  либо $H_1: \mu < \mu_0$

Еще у нас есть заранее заданный уровень значимости $\alpha$.

По имеющейся выборке мы находим среднее $M$, а стандартное отклонение $\sigma$ нам известно заранее. Тогда мы можем посчитать z-оценку для выборочного среднего в предположении, что верна нулевая гипотеза, эта величина должна быть распределена по нормальному закону $\frac{M - \mu_0}{\sigma / \sqrt{n}} \sim N(0;1)$

По уровню значимости $\alpha$ определяем критическую область: в случае двусторонней альтернативной гипотезы это будут два симметричных хвоста слева и справа по $\alpha /2$, а в случае лево- и правосторонней это соотвественно либо левый либо правый хвост.

Посмотрим, как это все реализованно в питоне: Вернемся к данным о первой контрольной и проверим одно из предположений о средней оценке

In [ ]:
from statsmodels.stats.weightstats import ztest


# Z-тест:
#   data - выборка
#   value - значение нулевой гипотезы
#   alternative - тип альтернативной гипотезы: 'two-sided', 'larger', 'smaller'

result = ztest(data1, value = 7.4, alternative = 'two-sided')

print(f'Z-statistics = {round(result[0],3)}\
        \n p-value = {round(result[1],3)}') # выводим значения z-cтатистики и p-value, оба значения округляем до 3 знаков


Z-statistics = -2.425        
 p-value = 0.015


Мы получили два числа, первое - это значение статистики, а второе, p-value, интерпретируется так: на любом уровне значимости, большем чем это число, нулевая гипотеза отвергается. В нашем случае: на уровнях значимости (например) 10%, 5%, 2% нулевая гипотеза отвергается, а на уровнях 1%, 0.5% и т.д не отвергается

Особенность реализации z-теста заключается в том, что в качестве генерального стандартного отклонения используется выборочное, что, впрочем, на больших выборках не играет роли.

ТЕОРЕТИЧЕСКОЕ ЗАДАНИЕ: попробуйте взять остальные варианты альтернативных гипотез, посмотрите на р-значения -- понимаете ли вы как они меняются? Что в случае односторонней альтернативы означает р-значение, близкое к 1?

Подставьте остальные предположения о средней оценке, посмотрите на получающиеся р-значения -- можно ли как-то заранее было определить, в каких случаях оно будет больше, а в каких меньше?

### Одновыборочный t-тест

Одновыборочный t-критерий тоже используется для проверки нулевой гипотезы о равенстве среднего значения $\mu$ генеральной совокупности, из которой была взята выборка, некоторому известному значению $\mu_0$. При этом у нас нет генерального стандартного отклонения.

$$H_0:\mu=\mu_0$$

В общем виде проверка (= тест) этой гипотезы выполняется при помощи t-критерия, который рассчитывается как отношение разницы между выборочным средним и известным значением к стандартной ошибке выборочного среднего:

$$t = \frac{M-μ_0}{s_M}$$
 
Рассчитанное значение критерия мы можем далее интерпретировать следующим образом, исходя из свойств t-распределения: если это значение попадает в т.н. область отклонения нулевой гипотезы (для двусторонней альтернативной гипотезы см. рисунок ниже), то мы вправе отклонить проверяемую нулевую гипотезу. Область отклонения нулевой гипотезы для критерия Стьюдента определяется заранее принятым уровнем значимости (например, $\alpha = 0.05$) и числом степеней свободы.

![image.png](https://www.researchgate.net/publication/324090392/figure/fig3/AS:609607676354560@1522353204155/T-test-area-of-acceptance-and-rejection-Source-Primary-data-processed-using-SPSS-2014.png)

Предположим, ботаник хочет знать, равна ли средняя высота определенного вида растения 15 дюймам. Она собирает случайную выборку из 12 растений и записывает их высоту в дюймах.

In [ ]:
data = [14, 14, 16, 13, 12, 17, 15, 14, 15, 13, 15, 14] #массив для хранения измерений 12 растений

Далее мы воспользуемся функцией ttest_1samp() из библиотеки scipy.stats для проведения одновыборочного t-теста:

In [ ]:
#perform one sample t-test
stats.ttest_1samp(a=data, popmean=15, alternative='two-sided')
#'less', 'greater' or 'two-sided', по умолчанию - двустороння

TtestResult(statistic=-1.6848470783484626, pvalue=0.12014460742498101, df=11)

Поскольку p-значение нашего теста (0,1201) больше, чем альфа = 0,05, мы не можем отвергнуть нулевую гипотезу теста. У нас нет достаточных доказательств, чтобы сказать, что средняя высота этого конкретного вида растений отличается от 15 дюймов.

### Парный t-тест

Теперь мы разберем, как проверять гипотезу о равенстве средних для парных (или связных) выборок. Такие выборки особенны тем, что каждому объекту из первой однозначно соответствует объект из другой. Например, это могут быть пары близнецов, где в каждой выборке по одному близнецу. Также такие выборки часто получаются при повторных измерениях. Предположим, мы проводим медицинское исследование, которое направлено на выявление эффективности препарата по снижению артериального давления. Тогда в первой выборке будут измерения давления до приема препарата, а во второй — после. Разумеется, объемы этих выборок всегда одинаковые.

Проверка гипотезы тогда строится вокруг разницы между одним измерением и другим, и нулевая гипотеза заключается в нулевой разнице, а альтернативная — в том, что разница не равна нулю.

Предположим, мы хотим знать, значительно ли влияет определенная учебная программа на успеваемость студента на конкретном экзамене. Чтобы проверить это, у нас есть 15 учеников в классе, которые проходят предварительный тест. Затем каждый из студентов участвует в учебной программе в течение двух недель. Затем учащиеся пересдают тест аналогичной сложности.

In [ ]:
pre = [88, 82, 84, 93, 75, 78, 84, 87, 95, 91, 83, 89, 77, 68, 91]
post = [91, 84, 88, 90, 79, 80, 88, 90, 90, 96, 88, 89, 81, 74, 92] #создадим массивы оценок до и после теста

Далее мы будем использовать функцию ttest_rel() из библиотеки scipy.stats для проведения t-теста парных выборок

In [ ]:
stats.ttest_rel(pre, post) # реализуем парный t-тест

TtestResult(statistic=-2.9732484231168796, pvalue=0.01007144862643272, df=14)

Статистика теста равна -2,9732 , а соответствующее двустороннее значение p равно 0,0101 .

В этом примере t-критерий парных выборок использует следующие нулевую и альтернативную гипотезы:

H 0 : Средние баллы до и после теста равны.

H A : Средние баллы до и после теста не равны.

Поскольку p-значение ( 0,0101 ) меньше 0,05, мы отвергаем нулевую гипотезу. У нас есть достаточно доказательств, чтобы сказать, что истинный средний балл теста отличается для студентов до и после участия в учебной программе.

### Двухвыборочный t-тест

При сравнении двух выборок проверяемая нулевая гипотеза состоит в том, что обе эти выборки происходят из нормально распределенных генеральных совокупностей с одинаковыми средними значениями:

$$H_0:\mu_1=\mu_0$$

Поскольку эти генеральные средние мы оцениваем при помощи выборочных средних значений, формула t-критерия приобретает вид

$$
\begin{align}
t = \frac{(M_1 - M_2) - (\mu_1 - \mu_2)}{s_{(M_1 - M_2)}}
\end{align}
$$

В знаменателе приведенной формулы находится стандартная ошибка разницы между выборочными средними, которая в общем виде рассчитывается как

$$
\begin{align}
s_{(M_1 - M_2)} = \sqrt{\frac{s^2_1}{n_1} + \frac{s^2_2}{n_2}}
\end{align}
$$

где $s_1^2$  и $s_2^2$
выборочные оценки дисперсии. При соблюдении условия о равенстве групповых дисперсий приведенная формула приобретает более простой вид. Интерпретация t-критерия, рассчитанного для двух выборок, выполняется точно так же, как и в случае с одной выборкой (см. выше).

Проведем t-тест для двух образцов, чтобы определить, имеют ли два вида растений одинаковую высоту.

In [ ]:
group1 = np.array([14, 15, 15, 16, 13, 8, 14, 17, 16, 14, 19, 20, 21, 15, 15, 16, 16, 13, 14, 12])
group2 = np.array([15, 17, 14, 17, 14, 8, 12, 19, 19, 14, 17, 22, 24, 16, 13, 16, 13, 18, 15, 13]) #создадим два массива для хранения измерений каждой группы из 20 растений 

воспользуемся функцией ttest_ind() из библиотеки scipy.stats для проведения двухвыборочного t-теста.

Прежде чем мы проведем тест, нам нужно решить, будем ли мы предполагать, что две совокупности имеют одинаковую дисперсию или нет (поправка Уэлча). Как правило, мы можем предположить, что совокупности имеют равные дисперсии, если отношение большей выборочной дисперсии к меньшей выборочной дисперсии составляет менее 4:1.

In [ ]:
print(np.var(group1), np.var(group2)) # находим дисперсии для обеих групп

7.727500000000001 12.260000000000002


Отношение большей дисперсии выборки к меньшей дисперсии выборки составляет 12,26 / 7,73 = 1,586 , что меньше 4. Это означает, что мы можем предположить, что дисперсии генеральной совокупности равны.

Таким образом, мы можем приступить к выполнению двухвыборочного t-критерия с равными дисперсиями:



In [ ]:
#perform two sample t-test with equal variances
stats.ttest_ind(a=group1, b=group2, equal_var=True)

Ttest_indResult(statistic=-0.6337397070250238, pvalue=0.5300471010405257)

Поскольку p-значение нашего теста (0,53005) больше, чем альфа = 0,05, мы не можем отвергнуть нулевую гипотезу теста. У нас нет достаточных данных, чтобы сказать, что средняя высота растений между двумя популяциями различна.

# Продажи и оценки видеоигр
Далее мы работаем с датасетом по продажам и оценкам видеоигр. Датасет взят с [Кеггла](https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings).

__Описание колонок:__
* `Name` $-$ название видеоигры
* `Platform` $-$ платформа, на которой игра была запущена
* `Year_of_Release` $-$ год релиза
* `Genre` $-$ жанр
* `Publisher` $-$ издатель
* `NA_Sales` $-$ объем продаж в Северной Америке (в млн штук)
* `EU_Sales` $-$ объем продаж в Евросоюзе (в млн штук)
* `JP_Sales` $-$ объем продаж в Японии (в млн штук)
* `Other_Sales` $-$ объем продаж в остальном мире (в млн штук)
* `Global_Sales` $-$ общий объем продаж (в млн штук)
* `Critic_Score` $-$ совокупный балл, составленный сотрудниками Metacritic
* `Critic_Count` $-$ кол-во критиков, оценивших игру
* `User_Score` $-$ совокупный балл, составленный подписчиками Metacritic (пользователями)
* `User_Count` $-$ кол-во пользователей, оценивших игру
* `Developer` $-$ ответственный за создание игры
* `Rating` $-$ рейтинг (Everyone, Teen, Adults Only и тд)



### ЗАДАНИЯ
По данным video_games_sales.csv
1. Выкинуть строки с пропусками, привести данные к правильным типам.
2. Построить 90% доверительный интервал для среднего объема продаж в Северной Америке, считая, что генеральное стандартное отклонение равно 1. Проинтерпретировать результат.
3. Построить 99% доверительный интервал для среднего объема продаж в Евросоюзе. Проинтерпретировать результат.
4. Считая, что генеральное стандартное отклонение равно выборочному, на уровне значимости 5% проверить гипотезу о том, что средний объем продаж в Японии равен 0.06, в качестве альтернативной использовать одностороннюю. На каких уровнях значимости будет отвергаться нулевая гипотеза?
5. На уровне значимости 1% проверить гипотезу о том, что средний общий объем продаж равен 0.9, в качестве альтернативной использовать двустороннюю.
6. На уровне значимости 5% проверить гипотезу о том, что средний объем продаж в Северной Америке равен среднему объему продаж в Евросоюзе в сумме с продажами в Японии и с продажами в остальном мире. Являются ли эти выборки связными или они независимы?
7. На уровне значимости 0.5% проверить гипотезу о том, средний общий объем продаж начиная с 2000 года такой же, как и до 2000.

In [ ]:
#задание 1
df = pd.read_csv('video_games_sales.csv')
df = df.dropna()
df['User_Score'] = df.User_Score.astype('float64')
df['Year_of_Release'] = df.Year_of_Release.astype('int64')
df['User_Count'] = df.User_Count.astype('int64')
df['Critic_Count'] = df.Critic_Count.astype('int64')

In [ ]:
df.info()

In [ ]:
df

In [ ]:
#задание 2
data1 = df['NA_Sales']
sigma = 1
stats.norm.interval(0.9, loc=data1.mean(), scale=sigma/np.sqrt(len(data1)))
# первый параметр это доверительная вероятность, второй - середина доверительного интервала, 
# третий - соответствующий множитель из формулы

С доверительной вероятностью \* данный интервал накрывает \*

In [ ]:
#задание 3
data2 = df['EU_Sales']
sigma = data2.std(ddof=1) # т.к это не генеральное стандартное отклонение, то делим не на n, а на n-1
stats.t.interval(0.99, df=len(data2) - 1, loc=data2.mean(), scale=sigma/np.sqrt(len(data2)))
# добавился новый параметр - это число степеней свободы, то есть n-1

In [ ]:
#задание 4
data1 = df['JP_Sales']
#   alternative - тип альтернативной гипотезы: 'two-sided', 'larger', 'smaller'

result = ztest(data1, value = 0.06, alternative = 'larger')

print(f'Z-statistics = {round(result[0],3)}\
        \n p-value = {round(result[1],3)}')


In [ ]:
#Задание 5
data = df['Global_Sales']
stats.ttest_1samp(a=data, popmean=0.9, alternative='two-sided')
#'less', 'greater' or 'two-sided', по умолчанию - двустороння

In [ ]:
#Задание 6
pre = df['NA_Sales']
post = df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales']
stats.ttest_rel(pre, post)